In [21]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta

In [16]:
usuario = 'maria.uribe@sansano.usm.cl'
token = '98c5510279d47e3e571b5750'
codigo_nacional = '330020'
year = 2025
month = 1

# API endpoint
url = f'https://climatologia.meteochile.gob.cl/application/servicios/getDatosRecientesEma/{codigo_nacional}/{year}/{month}?usuario={usuario}&token={token}'
url = f'https://climatologia.meteochile.gob.cl/application/servicios/getDatosRecientesEma/{codigo_nacional}/{year}/{month}'
print(url)
# Parameters for the API request
params = {
    'usuario': usuario,
    'token': token
}

# Make the request with SSL verification disabled
#response = requests.get(url, params=params, verify=False)

response = requests.get(url, params= params, verify=False)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()

https://climatologia.meteochile.gob.cl/application/servicios/getDatosRecientesEma/330020/2025/1


c:\Users\maria.uribe\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'climatologia.meteochile.gob.cl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [17]:
data

{'organismo': 'Dirección Meteorlógica de Chile',
 'pais': 'Chile',
 'fechaCreacion': '05-03-2025 16:54',
 'timezone': 'UTC',
 'producto': 'Datos Recientes de la estación automática últimas 12 horas',
 'registros': 2975,
 'status': 'Estación con datos disponibles recientemente',
 'datosEstaciones': {'estacion': {'codigoNacional': '330020',
   'codigoOMM': '85577',
   'codigoOACI': 'SCQN',
   'nombreEstacion': 'Quinta Normal, Santiago',
   'latitud': '-33.44500',
   'longitud': '-70.68278',
   'altura': 520},
  'datos': [{'momento': '2025-01-01 00:00:00',
    'temperatura': '24.4 °C',
    'temperatura02Mts': None,
    'temperatura10Mts': None,
    'temperatura30Mts': None,
    'puntoDeRocio': '11.3 °C',
    'temperaturaMinima12Horas': '19.8 °C',
    'temperaturaMaxima12Horas': '33.0 °C',
    'humedadRelativa': '43 %',
    'radiacionGlobalInst': ' 0.000 Watt/m2',
    'presionEstacion': '953.4 hPas.',
    'presionNivelDelMar': '1012.8 hPas.',
    'presionNivelEstandar': '1015.2 hPas.',
   

In [ ]:
def fetch_and_process_weather_data_for_period(start_date, end_date, usuario, token, codigo_nacional):
    # Create an empty DataFrame to store all the weather data
    all_weather_data = pd.DataFrame()
    # Loop over the period month-by-month
    current_date = start_date
    print(f'current date: {current_date}')
    while current_date <= end_date:
        print(f'end date: {end_date}')
        year = current_date.strftime('%Y')  # Get the current year
        month = current_date.strftime('%m')  # Get the current month

        # API endpoint
        url = f'https://climatologia.meteochile.gob.cl/application/servicios/getDatosRecientesEma/{codigo_nacional}/{year}/{month}'
        print(url)
        # Parameters for the API request
        params = {
            'usuario': usuario,
            'token': token
        }

        # Make the request with SSL verification disabled
        response = requests.get(url, params=params, verify=False)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()
            if 'datosEstaciones' in data and 'datos' in data['datosEstaciones']:
                weather_records = data['datosEstaciones']['datos']
                # Create a DataFrame from the weather records
                api_weather_df = pd.DataFrame(weather_records)
                # Convert 'momento' to a datetime object
                api_weather_df['momento'] = pd.to_datetime(api_weather_df['momento'])
                # Set the 'momento' column to UTC timezone (since the weather data is in UTC)
                api_weather_df['momento'] = api_weather_df['momento'].dt.tz_localize('UTC')
                # Convert the 'momento' column from UTC to Chilean local time
                api_weather_df['momento'] = api_weather_df['momento'].dt.tz_convert('America/Santiago')
                # Remove timezone information and keep only the datetime
                api_weather_df['momento'] = api_weather_df['momento'].dt.tz_localize(None)
                # Clean up and convert API fields to match historical DataFrame fields
                api_weather_df['Temperatura'] = api_weather_df['temperatura'].str.replace(' °C', '').astype(float)
                # Select relevant columns for further processing
                relevant_columns = ['momento', 'Temperatura']
                api_weather_df = api_weather_df[relevant_columns]
                # Set 'momento' as the index
                api_weather_df.set_index('momento', inplace=True)
                # Resample the data to hourly frequency (taking mean for continuous data)
                hourly_api_weather_df = api_weather_df.resample('H').mean()
                # Append to all_weather_data
                all_weather_data = pd.concat([all_weather_data, hourly_api_weather_df])
            else:
                print(f"Unexpected response structure: {data}")
        else:
            print(f"Failed to fetch data from API. Status code: {response.status_code}")
        # Move to the next month
        current_date += timedelta(days=32)
        current_date = current_date.replace(day=1)  # Reset to the first day of the next month
    return all_weather_data  # Return all the weather data for the period

In [24]:
today = datetime.now()
yesterday =today -timedelta(days=1)
tomorrow = today +timedelta(days=1)
usuario = 'maria.uribe@sansano.usm.cl'
token = '98c5510279d47e3e571b5750'
codigo_nacional = '330020'

api_weather_data = fetch_and_process_weather_data_for_period(usuario=usuario, token=token, codigo_nacional=codigo_nacional, start_date=yesterday, end_date=tomorrow)

end_date: 2025-03-06 13:59:48.018586
current date: 2025-03-04 13:59:48.018586
end date: 2025-03-06 13:59:48.018586
https://climatologia.meteochile.gob.cl/application/servicios/getDatosRecientesEma/330020/2025/03


c:\Users\maria.uribe\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'climatologia.meteochile.gob.cl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [25]:
api_weather_data

,Temperatura
momento,
2025-02-28 21:00:00,21.325
2025-02-28 22:00:00,19.750
2025-02-28 23:00:00,18.700
2025-03-01 00:00:00,17.425
2025-03-01 01:00:00,16.750
...,...
2025-03-05 09:00:00,18.700
2025-03-05 10:00:00,22.350
2025-03-05 11:00:00,25.025
